In [ ]:
import json

from sacrebleu import corpus_bleu, corpus_chrf

In [ ]:
# !pip install nltk
from nltk.translate import bleu, bleu_score

## Changes needed on the Ground truth dataset

Set of preprocessing hast to be done on both ground truth and predicted data, before applying metrics.

1. Update the DATE expressions if any
2. Remove the url with a fixed length text.
3. Ensure unncessary attributes are added " eg; AltText", aka.
4. Ensure the order key is same on both dataset

In [ ]:
data_gt = """{
	"$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
	"type": "AdaptiveCard",
	"version": "1.0",
	"body": [
		{
			"type": "Container",
			"items": [
				{
					"type": "TextBlock",
					"text": "Publish Adaptive Card schema",
					"weight": "bolder",
					"size": "medium"
				},
				{
					"type": "ColumnSet",
					"columns": [
						{
							"type": "Column",
							"width": "auto",
							"items": [
								{
									"type": "Image",
									"url": "imageURL",
									"size": "small",
									"style": "person"
								}
							]
						},
						{
							"type": "Column",
							"width": "stretch",
							"items": [
								{
									"type": "TextBlock",
									"text": "Matt Hidinger",
									"weight": "bolder",
									"wrap": true
								},
								{
									"type": "TextBlock",
									"spacing": "none",
									"text": "Created Tue, 14 Feb 2017",
									"isSubtle": true,
									"wrap": true
								}
							]
						}
					]
				}
			]
		},
		{
			"type": "Container",
			"items": [
				{
					"type": "TextBlock",
					"text": "Now that we have defined the main rules and features of the format, we need to produce a schema and publish it to GitHub. The schema will be the starting point of our reference documentation.",
					"wrap": true
				},
				{
					"type": "FactSet",
					"facts": [
						{
							"title": "Board:",
							"value": "Adaptive Card"
						},
						{
							"title": "List:",
							"value": "Backlog"
						},
						{
							"title": "Assigned to:",
							"value": "Matt Hidinger"
						},
						{
							"title": "Due date:",
							"value": "Not set"
						}
					]
				}
			]
		}
	],
	"actions": [
		{
			"type": "Action.ShowCard",
			"title": "Set due date",
			"card": {
				"type": "AdaptiveCard",
				"body": [
					{
						"type": "Input.Date",
						"id": "dueDate",
						"title": "Select due date"
					},
					{
						"type": "Input.Text",
						"id": "comment",
						"isMultiline": true,
						"placeholder": "Add a comment"
					}
				],
				"actions": [
				    {
				        "type": "Action.OpenUrl",
						"title": "OK",
						"url": "https://adaptivecards.io"
			        }
				]
			}
		},
		{
			"type": "Action.OpenUrl",
			"title": "View",
			"url": "https://adaptivecards.io"
		}
	]
}"""

data_pred = """{
  "type": "AdaptiveCard",
  "version": "1.0",
  "body": [
    {
      "type": "TextBlock",
      "text": "Publish Adaptive Card schema",
      "size": "Default",
      "horizontalAlignment": "Left",
      "color": "Dark",
      "weight": "Bolder",
      "wrap": "true"
    },
    {
      "type": "ColumnSet",
      "columns": [
        {
          "type": "Column",
          "width": "auto",
          "items": [
            {
              "type": "Image",
              "altText": "Image",
              "horizontalAlignment": "Left",
              "size": "Small",
              "url": "imgURL"
            }
          ],
          "horizontalAlignment": "Right"
        },
        {
          "type": "Column",
          "width": "auto",
          "items": [
            {
              "type": "TextBlock",
              "text": "Matt Hidinger",
              "size": "Default",
              "horizontalAlignment": "Left",
              "color": "Dark",
              "weight": "Bolder",
              "wrap": "true"
            },
            {
              "type": "TextBlock",
              "text": "Created Tue, 14 Feb 2017",
              "size": "Default",
              "horizontalAlignment": "Left",
              "color": "Dark",
              "weight": "Lighter",
              "wrap": "true"
            }
          ],
          "horizontalAlignment": "Right"
        }
      ],
      "horizontalAlignment": "Left"
    },
    {
      "type": "TextBlock",
      "text": "Now that we have defined the main rules and features of the format, we need to produce a schema and publish it to GitHub. The schema will be the starting point of our reference documentation.",
      "size": "Default",
      "horizontalAlignment": "Left",
      "color": "Dark",
      "weight": "Default",
      "wrap": "true"
    },
    {
      "type": "ColumnSet",
      "columns": [
        {
          "type": "Column",
          "width": "auto",
          "items": [
            {
              "type": "TextBlock",
              "text": "Board:",
              "size": "Default",
              "horizontalAlignment": "Left",
              "color": "Dark",
              "weight": "Default",
              "wrap": "true"
            }
          ],
          "horizontalAlignment": "Right"
        },
        {
          "type": "Column",
          "width": "auto",
          "items": [
            {
              "type": "TextBlock",
              "text": "Adaptive Card",
              "size": "Default",
              "horizontalAlignment": "Left",
              "color": "Dark",
              "weight": "Default",
              "wrap": "true"
            }
          ],
          "horizontalAlignment": "Right"
        }
      ],
      "horizontalAlignment": "Left"
    },
    {
      "type": "TextBlock",
      "text": "Assigned to: Matt Hidinger",
      "size": "Default",
      "horizontalAlignment": "Left",
      "color": "Dark",
      "weight": "Default",
      "wrap": "true"
    },
    {
      "type": "TextBlock",
      "text": "Due date: Not set",
      "size": "Default",
      "horizontalAlignment": "Left",
      "color": "Dark",
      "weight": "Default",
      "wrap": "true"
    },
    {
      "type": "ActionSet",
      "actions": [
        {
          "type": "Action.Submit",
          "title": "Set due date",
          "style": "default"
        }
      ],
      "spacing": "Medium",
      "horizontalAlignment": "Left"
    },
    {
      "type": "ActionSet",
      "actions": [
        {
          "type": "Action.Submit",
          "title": "Comment",
          "style": "default"
        }
      ],
      "spacing": "Medium",
      "horizontalAlignment": "Left"
    }
  ],
  "$schema": "http://adaptivecards.io/schemas/adaptive-card.json"
}
"""

In [ ]:
def update_img_url(schema):
    "Find element type 'Image' and change url to 'image'"
    
    
    pass

# json.dumps(json.loads(data_gt)).split()

## Metrics to Measure the quality of the Pic2Card pipeline


We can employee two types of methods

1. Evaluate the generated pic2card schema to the source schema
2. Compare the source image with the image gnerated from the predicted schema.



### Down sides of seq-2-seq comparison method

1. Chance of missing some prediction by model, may cause alignment issues while matching the sequences.

> What happens if a model missed some element in bbox ?

> What to do with the image, how we compare it ?

> 



2. Text based metho,

ROUGE and BLUE - https://www.slideshare.net/hamidshahrivari/bleu-vs-rouge



## Issues with using BLEU

BLEU was designed for the Machine Translation, which means the sentences are compared against human translated sentence. That means, the human sentences may not have all words same even though the translation is accurate. Blue is in place to capture these nuances with MT. In order to do that BLEU expects more than one reference setntences to compare against a candidate sentence, so that the score can be averaged out due to the different human can use different sentence structure or words.

At last the matching is done n-gram basis matches, with tuning parameters to manage the candidate strings is too small or larger than the reference sentences.

Pic2card scenario, the scenario is not exactly like MT, but we can correlate though as both are trying generate sequence and want to ensure both follow exact syntax. In pic2card the ways in which the sequence can be formed is very restricted not like natural language, there is only one way is allowed.

So we can leverage the techniques of BLEU like n-gram matching, but not all the methodlogies of BLEU. So with n-gram and 
jaccard similiary to see both sentences are matching is good measure to understand the similarity or closeness with the
predicted pic2card schema vs. ground truth one.

## The main reasons why ROUGE isn't used much

In MT space mainly used matrics is BLEU, the issues with using ROUTE the recall based metrics is that, since BLEU metrics are relayed up on the multiple reference sentences from different humans to compare a translated sentence. That getting recall for each human sentence isn't that practical measure, as in this case all the translations are good. This difficulty is leads to the recall based measure isn't that friendly with the Machine Translation based problems.

In [ ]:
corpus_bleu([data_pred], data_gt)

In [ ]:
# data_pred
bleu = corpus_bleu("hello i'm haridas", "hello i'm asdf")

In [ ]:
# bleu_score.sentence_bleu("hello i'm haridas ad".split(), "hello i'm asdf ad".split())
# json.loads(data_gt)

In [ ]:
# corpus_bleu?
>>> hyp1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
...         'ensures', 'that', 'the', 'military', 'always',
...         'obeys', 'the', 'commands', 'of', 'the', 'party']
>>> ref1a = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
...          'ensures', 'that', 'the', 'military', 'will', 'forever',
...          'heed', 'Party', 'commands']
>>> ref1b = ['It', 'is', 'the', 'guiding', 'principle', 'which',
...          'guarantees', 'the', 'military', 'forces', 'always',
...          'being', 'under', 'the', 'command', 'of', 'the', 'Party']
>>> ref1c = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
...          'army', 'always', 'to', 'heed', 'the', 'directions',
...          'of', 'the', 'party']

>>> hyp2 = ['he', 'read', 'the', 'book', 'because', 'he', 'was',
...         'interested', 'in', 'world', 'history']
>>> ref2a = ['he', 'was', 'interested', 'in', 'world', 'history',
...          'because', 'he', 'read', 'the', 'book']

In [ ]:
smoothing_fns = bleu_score.SmoothingFunction()

smoothing_fns.method0

In [ ]:
bleu_score.sentence_bleu([ref1a], hyp1)

In [ ]:
# bleu_score.sentence_bleu([ref1a], hyp1, smoothing_function=smoothing_fns.method7)

In [ ]:
bleu_score.sentence_bleu([data_gt.split()], data_pred.split(), smoothing_function=smoothing_fns.method7)